In [1]:
import pandas as pd

import numpy as np
# Import module for Fuzzy string matching
from fuzzywuzzy import fuzz, process
# Import module for regex
import re
# Import module for iteration
import itertools
# Import module for function development
from typing import Union, List, Tuple
# Import module for TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
# Import module for cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
# Import module for KNN
from sklearn.neighbors import NearestNeighbors

import time


# String pre-processing
def preprocess_string(s):
    # Remove spaces between strings with one or two letters
    s = re.sub(r'(?<=\b\w)\s*[ &]\s*(?=\w\b)', '', s)
    return s

# String matching - TF-IDF
def build_vectorizer(
    clean: pd.Series,
    analyzer: str = 'char', 
    ngram_range: Tuple[int, int] = (1, 4), 
    n_neighbors: int = 1, 
    **kwargs
    ) -> Tuple:
    # Create vectorizer
    vectorizer = TfidfVectorizer(analyzer = analyzer, ngram_range = ngram_range, **kwargs)
    X = vectorizer.fit_transform(clean.values.astype('U'))

    # Fit nearest neighbors corpus
    nbrs = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(X)
    return vectorizer, nbrs

# String matching - KNN
def tfidf_nn(
    messy, 
    clean, 
    n_neighbors = 1, 
    **kwargs
    ):
    # Fit clean data and transform messy data
    vectorizer, nbrs = build_vectorizer(clean, n_neighbors = n_neighbors, **kwargs)
    input_vec = vectorizer.transform(messy)

    # Determine best possible matches
    distances, indices = nbrs.kneighbors(input_vec, n_neighbors = n_neighbors)
    nearest_values = np.array(clean)[indices]
    return nearest_values, distances

# String matching - match fuzzy
def find_matches_fuzzy(
    row, 
    match_candidates, 
    limit = 5
    ):
    row_matches = process.extract(
        row, dict(enumerate(match_candidates)), 
        scorer = fuzz.token_sort_ratio, 
        limit = limit
        )
    result = [(row, match[0], match[1]) for match in row_matches]
    return result

# String matching - TF-IDF
def fuzzy_nn_match(
    messy,
    clean,
    column,
    col,
    n_neighbors = 100,
    limit = 5, **kwargs):
    nearest_values, _ = tfidf_nn(messy, clean, n_neighbors, **kwargs)

    results = [find_matches_fuzzy(row, nearest_values[i], limit) for i, row in enumerate(messy)]
    df = pd.DataFrame(itertools.chain.from_iterable(results),
        columns = [column, col, 'Ratio']
        )
    return df

# String matching - Fuzzy
def fuzzy_tf_idf(
    df: pd.DataFrame,
    column: str,
    clean: pd.Series,
    mapping_df: pd.DataFrame,
    col: str,
    analyzer: str = 'char',
    ngram_range: Tuple[int, int] = (1, 3)
    ) -> pd.Series:
    # Create vectorizer
    clean = clean.drop_duplicates().reset_index(drop = True)
    messy_prep = df[column].drop_duplicates().dropna().reset_index(drop = True).astype(str)
    messy = messy_prep.apply(preprocess_string)
    result = fuzzy_nn_match(messy = messy, clean = clean, column = column, col = col, n_neighbors = 1)
    # Map value from messy to clean
    return result

/home/gmartini2019/miniconda3/envs/SherlockEnv/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Error processing line 1 of /home/gmartini2019/miniconda3/envs/SherlockEnv/lib/python3.7/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/home/gmartini2019/miniconda3/envs/SherlockEnv/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored


In [2]:
df = pd.read_csv('/mnt/c/Users/marti/OneDrive/Desktop/room_type.csv')
# Check the dimension of data
print('Dimension of data: {} rows and {} columns'.format(len(df), len(df.columns)))
# Print the first 5 rows
df_result = (df.pipe(fuzzy_tf_idf, # Function and messy data
                     column = 'Expedia', # Messy column in data
                     clean = df['Booking.com'], # Master data (list)
                     mapping_df = df, # Master data
                     col = 'Result') # Can be customized
            )
df_result.head()

Dimension of data: 103 rows and 2 columns


,Expedia,Result,Ratio
0,"Deluxe Room, 1 King Bed",Deluxe Room - One King Bed,87
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible,78
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,80
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite,85
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room,90


In [3]:
def stringMatching(
    df: pd.DataFrame,
    column: str,
    clean: pd.Series,
    mapping_df: pd.DataFrame,
    col: str
    ):
    # Create vectorizer
    categoryClean = clean.drop_duplicates().reset_index(drop = True)
    categoryMessy = df[column].drop_duplicates().dropna().reset_index(drop = True).astype(str)

    categoryFuzzy = {}
    ratioFuzzy = {}
    for i in range(len(categoryMessy)):
        resultFuzzy = process.extractOne(categoryMessy[i].lower(), categoryClean)
        # Mapping
        catFuzzy = {categoryMessy[i]:resultFuzzy[0]}
        ratFuzzy = {categoryMessy[i]:resultFuzzy[1]}
        # Save result
        categoryFuzzy.update(catFuzzy)
        # Save the ratio
        ratioFuzzy.update(ratFuzzy)

    # Create column names
    catCol = col
    ratCol = 'Ratio'
    # Merge the result
    df[catCol] = df[column]
    df[ratCol] = df[column]
    # Mapping the result
    df[catCol] = df[catCol].map(categoryFuzzy)
    df[ratCol] = df[ratCol].map(ratioFuzzy)
    return df



df_result = (df.pipe(stringMatching, # Function and messy data
                     column = 'Expedia', # Messy column in data
                     clean = df['Booking.com'], # Master data (list)
                     mapping_df = df, # Master data
                     col = 'Result') # Can be customized
            )
df_result.head()

,Expedia,Booking.com,Result,Ratio
0,"Deluxe Room, 1 King Bed",Deluxe King Room,Deluxe King Room,95
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible,Deluxe King Room,86
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,Grand Corner King Room,90
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite,King Parlor Suite,95
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room,High-Floor Premium King Room,95


In [12]:
cols = ['Clean', 'Messy']
test_data = pd.DataFrame(index = np.arange(7), columns= cols)

Clean = ['Street', 'Address', 'First Name', 'Last Name', 'Telephone Number', 'Credit Card Number', 'Room Type']
test_data['Clean'] = Clean

Messy = ['Strt', 'Addr', 'Personal Name', 'Family Name', 'Telephone Num', 'Cred Card Num', 'Room T']
test_data['Messy'] = Messy

test_data

,Clean,Messy
0,Street,Strt
1,Address,Addr
2,First Name,Personal Name
3,Last Name,Family Name
4,Telephone Number,Telephone Num
5,Credit Card Number,Cred Card Num
6,Room Type,Room T


In [13]:
df_result = (test_data.pipe(stringMatching, # Function and messy data
                     column = 'Messy', # Messy column in data
                     clean = test_data['Clean'], # Master data (list)
                     mapping_df = test_data, # Master data
                     col = 'Result') # Can be customized
            )
df_result.head()

,Clean,Messy,Result,Ratio
0,Street,Strt,Street,80
1,Address,Addr,Address,90
2,First Name,Personal Name,First Name,61
3,Last Name,Family Name,First Name,67
4,Telephone Number,Telephone Num,Telephone Number,90
